<a href="https://www.kaggle.com/code/pogscafe/retrieval-based-voice-conversion-rvc-webui?scriptVersionId=174269932" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# RVC WebUI

How-to video: https://youtu.be/SJUw6evAXbY  
Repository: https://github.com/wandaweb/RVC-WebUI-Kaggle  
    
Check out the original RVC project: https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI  
and the WebUI repo: https://github.com/ddPn08/rvc-webui

If you want to help me dedicate more time to creating and updating notebooks like this one, consider joining on [Pateron](patreon.com/PogsCafe), or [Buy me a Coffee](https://ko-fi.com/pogscafe)! Get early previews of notebooks, videos, prompts, and end-of-video shoutouts as soon as we reach 10 members!

## Installation
Installation can take about 4 minutes the first time, and about 3 minutes in consecutive runs.

In [ ]:
%cd /kaggle/working

!git clone https://github.com/wandaweb/Retrieval-based-Voice-Conversion-WebUI
%cd Retrieval-based-Voice-Conversion-WebUI
!pip install -q -r requirements.txt
!python download-models-for-kaggle.py

%cd /kaggle/working
!git clone https://github.com/ddPn08/rvc-webui
%cd rvc-webui

%cd /kaggle/working/rvc-webui

import fileinput
import os
import launch
    
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

launch.prepare_environment()
os.environ["PATH"] = (
        "/kaggle/working/rvc-webui/bin"
        + os.pathsep
        + os.environ.get("PATH", "")
    )

!pip install -q  --force-reinstall --no-deps tensorboard
!pip install -q  --force-reinstall --no-deps numpy==1.23.5
!pip install -q  --force-reinstall --no-deps wandb==0.15.11
!pip install -q  transformers==4.28.0

%cd /kaggle/working/rvc-webui
if not os.path.exists(f"/kaggle/working/rvc-webui/linux-amd64-filebrowser.tar.gz"):
    get_ipython().system("wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz")
if not os.path.exists(f"/kaggle/working/rvc-webui/filebrowser"):   
    get_ipython().system("tar xvfz linux-amd64-filebrowser.tar.gz")
!chmod a+x /kaggle/working/rvc-webui/filebrowser
!/kaggle/working/rvc-webui/filebrowser config init > /dev/null
!/kaggle/working/rvc-webui/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/working/rvc-webui/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/working/rvc-webui/filebrowser users add admin admin 
!/kaggle/working/rvc-webui/filebrowser config export "/kaggle/working/config.json"

### Download a song

In [ ]:
# Example: Download a song
# The song I'm using in this example is Anything Can Happen (4:06) by Geoff Hurley
# available on https://www.freemusicpublicdomain.com/royalty-free-ballads/
!mkdir /kaggle/working/rvc-webui/song/
!wget "https://www.dropbox.com/s/6zg5sietcjit9lt/08%20-%20Geoff%20Hurley%20-%20Anything%20Can%20Happen.mp3?dl=1" \
    -O "/kaggle/working/rvc-webui/song/AnythingCanHappen.mp3"

## Split an audio file into vocal and instrumental tracks
If you have a song with background music and vocals, or if you have a voice recording with background noise, you can use this step to extract the vocals into a separate file.  
Change the value of the UVR_FILE variable to point to the audio file you want to split.

In [ ]:
# Extract voice
os.environ["UVR_FILE"] = "/kaggle/working/rvc-webui/song/AnythingCanHappen.mp3"

os.environ["UVR_MODEL"] = "HP5_only_main_vocal"
os.environ["UVR_OUT"] = "opt"
os.environ["UVR_FORMAT"] = "wav"

os.environ["weight_uvr5_root"] = "/kaggle/working/Retrieval-based-Voice-Conversion-WebUI/assets/uvr5_weights"
os.environ["TEMP"] = "/tmp"

%cd /kaggle/working/Retrieval-based-Voice-Conversion-WebUI

!python extract.py

## Run RVC WebUI

**Option 1: Starting the Web UI Pinggy**  
* Wait for the line that says "Running on local URL:  http://127.0.0.1:7860". It might take a minute for this line to show up at the end of the output.   
* Click the link ending with pinggy.link between the smiling emojis

* If your (training) code is still running when your link time runs out in an hour, wait for the training to complete, and re-run this code block to interact with the UI again

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = "python -c 'import launch; launch.start()' & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7860 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            #print(line)
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

**Option 2: Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "Running on local URL:  http://127.0.0.1:7860" 
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# Starting the Web UI with ngrok

# --- Variables ---

Ngrok_token = "" 
# Put your ngrok token here (obtainable from https://ngrok.com)
# Example: Ngrok_token = "2Fw13n4GcJT12g7mSDUC62cdNGb_5svdjf3Gg5vfhr4nGr5gF"

Ngrok_domain = "" # optional, leave empty if you don't have a domain

port = 7860

# -----------------


!pip install pyngrok

code = "import launch; launch.start()"

from pyngrok import ngrok, conf
import fileinput
import sys
import gc

gc.collect()

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(port , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print("\n😁* Your URL is: *😁")
  print(srv)
  print ("😁****************😁\n")
  get_ipython().system(f"python  -c '{code}'")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

**Option 3: Starting the Web UI with Remote Moe - Warning: Will not work if port 22 is blocked, which is the case on Kaggle environments since April 26 2024. I'm leaving this code here in hope this will change.**  

In [ ]:
# Starting the Web UI with Remote Moe - WARNING: Kaggle recently started blocking port 22, so this option does not work at the moment

%cd /kaggle/working/rvc-webui
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python -c "import launch; import warnings; warnings.filterwarnings('ignore'); launch.start()" & ssh -R 80:127.0.0.1:7860 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

### Run FileBrowser 

In [ ]:
%cd /kaggle/working
!chmod a+x /kaggle/working/rvc-webui/filebrowser
        
from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    !/kaggle/working/rvc-webui/filebrowser -c "/kaggle/working/config.json" & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8080 a.pinggy.io > log.txt > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()


## Download a model

In [ ]:
pth_url = "https://huggingface.co/sail-rvc/SpongeBob_SquarePants__RVC_v2_/resolve/main/model.pth?download=true"
index_url = "https://huggingface.co/sail-rvc/SpongeBob_SquarePants__RVC_v2_/resolve/main/model.index?download=true"
name = "spongebob"


get_ipython().system(f'wget "{pth_url}" -O "/kaggle/working/rvc-webui/models/checkpoints/{name}.pth"')
index_folder = f'/kaggle/working/rvc-webui/models/checkpoints/{name}_index'
get_ipython().system(f'mkdir "{index_folder}"')
get_ipython().system(f'wget "{index_url}" -O "{index_folder}/{name}.index"')



## Merge two audio tracks (warning: this sometimes crashes the runtime - make sure to save your outputs first)

In [ ]:
audio1 = "/kaggle/working/rvc-webui/outputs/2-spongebob-vocal_anything.mp3_10.wav"
audio2 = "/kaggle/working/Retrieval-based-Voice-Conversion-WebUI/opt/instrument_anything.mp3_10.wav"
out = "/kaggle/working/audio-out/spongebob.wav"

!ls -la $audio1
!ls -la $audio2

!mkdir /kaggle/working/audio-out

!ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i $audio1 -i $audio2 -c:v copy -filter_complex "[0:a][1:a] amix=inputs=2:duration=longest" $out

## Delete checkpoints
Delete all trained models to free up space.

In [ ]:
!rm -rf /kaggle/working/rvc-webui/models/checkpoints/*

## Delete everything in /kaggle/working
Useful if you want to run a clean installation or free up space without manually deleting models you no longer need. Make sure to download your outputs first!

In [ ]:
!rm -rf /kaggle/working/*